In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# 本章第一节课的案例数据
x1 = [1,2,4,5,7,8,9,3,6]
x2 = [2,2,5,7,1,3,5,2,6]
data = np.array([i for i in zip(x1,x2)])
data

array([[1, 2],
       [2, 2],
       [4, 5],
       [5, 7],
       [7, 1],
       [8, 3],
       [9, 5],
       [3, 2],
       [6, 6]])

### K means

In [5]:
def distance(u,v):
    '''
    u,v: 一维 numpy 数组表示的特征向量
    '''
    # 返回欧氏距离的平方
    return sum((u-v)**2)

import numpy as np

# 从 data 中随机选择 k 个样本特征向量，作为初始聚类中心
def center_init(data,k):
    '''
    data: 存放样本特征向量的 numpy array
    k: 聚类中心的个数
    '''
    # 随机选择 k 个样本特征向量的索引
    index = np.random.choice(a=range(len(data)),size=k)
    # 根据索引获得 k 个聚类中心
    return data[index]

def predict(x,center):
    '''
    x: 一维数组表示的样本特征向量
    center: 存放 k 个聚类中心的数组
    '''
    # 输入的样本 x 必须为一维数组，否则触发异常
    assert np.array(x).ndim==1
    # 返回距离样本 x 最近的聚类中心索引
    return np.argmin([distance(x,c) for c in center])

def cluster(data,center):
    '''
    data: 存放样本特征向量的 numpy array
    center: 存放 k 个聚类中心的数组
    '''
    # 存放各类样本子集的字典：key = 聚类中心索引; value = data 子集
    center_data = dict()
    # 遍历样本特征向量
    for x in data:
        # 获取样本 x 的聚类索引
        i =  predict(x,center)
        # 将 x 添加到聚类索引对应的样本子集中
        center_data.setdefault(i,[])
        center_data[i].append(x)
    # 返回各聚类中心对应的样本子集
    return center_data

def fit(data,k):
    '''
    data: 存放样本特征向量的 numpy array
    visualization： 可视化选项，默认为 None，不对聚类过程做可视化
    '''
    # 初始化聚类中心
    center = center_init(data,k)
    # 循环迭代
    while True:
        # 根据已有的聚类中心将样本划分到不同的聚类集合中
        center_data = cluster(data,center)
        old_center = center.copy()
        # 遍历各个聚类中心的索引
        for i in center_data:
            # 更新每一个聚类中心：样本子集特征向量的均值
            center[i] = np.mean(center_data[i],axis=0)
        # 循环迭代的停止条件：最近两次迭代，聚类中心的位置不再变化
        if sum([distance(center[i],old_center[i]) for i in range(k)]) == 0:
            break
    return center,center_data

In [6]:
# 聚类簇数
k = 3
# 获取最终的聚类中心和对应的样本子集，并对聚类过程可视化
center,center_data = fit(data,k)

In [7]:
center

array([[8, 3],
       [2, 2],
       [5, 6]])

In [8]:
center_data

{1: [array([1, 2]), array([2, 2]), array([3, 2])],
 2: [array([4, 5]), array([5, 7]), array([6, 6])],
 0: [array([7, 1]), array([8, 3]), array([9, 5])]}

### K means ++

In [11]:
def distance(u,v):
    '''
    u,v: 一维 numpy 数组表示的特征向量
    '''
    # 返回欧氏距离的平方
    return sum((u-v)**2)

import numpy as np

import numpy as np 

def center_init(data,k):
    '''
    data: numpy array,样本特征向量集合
    k: int,聚类中心的个数
    '''
    # 用来保存聚类中心
    center = []
    # 随机选择一个样本特征向量的索引
    index = np.random.choice(range(len(data)))
    # 根据索引获得第一个聚类中心
    center.append(data[index])
    # 获取剩余 k-1 个聚类中心
    for _ in range(k-1):
        # 样本被选为聚类中心的概率
        prob = []
        # 遍历每个样本数据
        for x in data:
            # 根据步骤 2 的公式计算样本与最近类中心的距离
            d = min([distance(x,c) for c in center])
            prob.append(d)
        # 计算每个样本的被选概率
        prob = np.array(prob)/sum(prob)
        # 根据被选中的概率随机选择一个样本特征向量的索引
        i = np.random.choice(a=range(len(data)),p=prob)
        # 根据索引添加一个聚类中心
        center.append(data[i])
    # 返回 k 个初始化的聚类中心
    return np.array(center)

def predict(x,center):
    '''
    x: 一维数组表示的样本特征向量
    center: 存放 k 个聚类中心的数组
    '''
    # 输入的样本 x 必须为一维数组，否则触发异常
    assert np.array(x).ndim==1
    # 返回距离样本 x 最近的聚类中心索引
    return np.argmin([distance(x,c) for c in center])

def cluster(data,center):
    '''
    data: 存放样本特征向量的 numpy array
    center: 存放 k 个聚类中心的数组
    '''
    # 存放各类样本子集的字典：key = 聚类中心索引; value = data 子集
    center_data = dict()
    # 遍历样本特征向量
    for x in data:
        # 获取样本 x 的聚类索引
        i =  predict(x,center)
        # 将 x 添加到聚类索引对应的样本子集中
        center_data.setdefault(i,[])
        center_data[i].append(x)
    # 返回各聚类中心对应的样本子集
    return center_data

def fit(data,k):
    '''
    data: 存放样本特征向量的 numpy array
    visualization： 可视化选项，默认为 None，不对聚类过程做可视化
    '''
    # 初始化聚类中心
    center = center_init(data,k)
    # 循环迭代
    while True:
        # 根据已有的聚类中心将样本划分到不同的聚类集合中
        center_data = cluster(data,center)
        old_center = center.copy()
        # 遍历各个聚类中心的索引
        for i in center_data:
            # 更新每一个聚类中心：样本子集特征向量的均值
            center[i] = np.mean(center_data[i],axis=0)
        # 循环迭代的停止条件：最近两次迭代，聚类中心的位置不再变化
        if sum([distance(center[i],old_center[i]) for i in range(k)]) == 0:
            break
    return center,center_data

In [12]:
# 聚类簇数
k = 3
# 获取最终的聚类中心和对应的样本子集，并对聚类过程可视化
center,center_data = fit(data,k)

In [13]:
center

array([[6.31458333, 2.89583333, 4.97395833, 1.703125  ],
       [5.17575758, 3.62424242, 1.47272727, 0.27272727],
       [4.73809524, 2.9047619 , 1.79047619, 0.35238095]])

In [14]:
data

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1],
       [5.4, 3.7, 1.5, 0.2],
       [4.8, 3.4, 1.6, 0.2],
       [4.8, 3. , 1.4, 0.1],
       [4.3, 3. , 1.1, 0.1],
       [5.8, 4. , 1.2, 0.2],
       [5.7, 4.4, 1.5, 0.4],
       [5.4, 3.9, 1.3, 0.4],
       [5.1, 3.5, 1.4, 0.3],
       [5.7, 3.8, 1.7, 0.3],
       [5.1, 3.8, 1.5, 0.3],
       [5.4, 3.4, 1.7, 0.2],
       [5.1, 3.7, 1.5, 0.4],
       [4.6, 3.6, 1. , 0.2],
       [5.1, 3.3, 1.7, 0.5],
       [4.8, 3.4, 1.9, 0.2],
       [5. , 3. , 1.6, 0.2],
       [5. , 3.4, 1.6, 0.4],
       [5.2, 3.5, 1.5, 0.2],
       [5.2, 3.4, 1.4, 0.2],
       [4.7, 3.2, 1.6, 0.2],
       [4.8, 3.1, 1.6, 0.2],
       [5.4, 3.4, 1.5, 0.4],
       [5.2, 4.1, 1.5, 0.1],
       [5.5, 4.2, 1.4, 0.2],
       [4.9, 3

You can also write the center_init as

In [15]:
import numpy as np

def center_init(data,k):
    '''
    data: numpy array,样本特征向量集合
    k: int,聚类中心的个数
    '''
    # 用来保存聚类中心
    center = []
    # 随机选择一个样本特征向量的索引
    index = np.random.choice(range(len(data)))
    # 根据索引获得第一个聚类中心
    center.append(data[index])
     # 获取剩余 k-1 个聚类中心
    for _ in range(k-1):
        # 存放样本与最近聚类中心的距离
        arr = []
         # 遍历每个样本数据
        for x in data:
            # 根据步骤 2 的公式计算样本与最近类中心的距离
            d = min([distance(x,c) for c in center])
            arr.append(d)
        # 选择 d 值最大的样本索引
        i = np.argmax(arr)
        # 根据索引添加一个聚类中心
        center.append(data[i])
    # 返回 k 个初始化的聚类中心
    return np.array(center)

#### Class format K means

In [16]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from sklearn.datasets import load_iris

class Kmeans:
    
    def __init__(self,k):
        ''' 初始化实例属性
        
        Parameters
        ----------
        k: 聚类中心的个数
        '''
        self.k = k
        # 存放 k 个聚类中心的数组
        self.center = np.array([])
        # 存放聚类样本子集的字典
        self.center_data = dict()
    
    def distance(self,u,v):
        ''' 样本距离的定义
        
        Parameters
        ----------
        u,v: 一维 numpy 数组表示的特征向量
        '''
        # 返回欧氏距离的平方
        return sum((u-v)**2)
    
    def center_init(self,data):
        ''' 初始化聚类中心
        
        从 data 中随机选择 k 个样本特征向量，作为初始聚类中心
        
        Parameters
        ----------
        data: 存放样本特征向量的 numpy array
        '''
        # 随机选择 k 个样本特征向量的索引
        index = np.random.choice(a=range(len(data)),size=self.k)
        # 根据索引获得 k 个聚类中心
        return data[index]
    
    def predict(self,x):
        ''' 获取样本 x 的聚类索引
        
        Parameters
        ----------
        x: 一维数组表示的样本特征向量
        '''
        # 输入的样本 x 必须为一维数组，否则触发异常
        assert np.array(x).ndim==1
        # 返回距离样本 x 最近的聚类中心索引
        return np.argmin([self.distance(x,c) for c in self.center])
    
    def cluster(self,data):
        ''' 获取聚类中心对应的样本子集
        
        Parameters
        ----------
        data: 存放样本特征向量的 numpy array
        '''
        # 存放各类样本子集的字典：key = 聚类中心索引; value = data 子集
        center_data = dict()
        # 遍历样本特征向量
        for x in data:
            # 获取样本 x 的聚类索引
            i =  self.predict(x)
            # 将 x 添加到聚类索引对应的样本子集中
            center_data.setdefault(i,[])
            center_data[i].append(x)
        # 返回各聚类中心对应的样本子集
        return center_data
    
    def visualization_2D(self):
        ''' 二维空间可视化'''
        # 设置画布大小
        plt.figure(figsize=(8,6))
        # 设置可选颜色：红、绿、蓝
        color = ['r','g','b']
        # 遍历每个聚类中心的索引
        for i in self.center_data:
            # 根据索引，取该聚类中心对应的样本第一列特征
            x = np.array(self.center_data[i])[:,0]
            # 根据索引，取该聚类中心对应的样本第二列特征
            y = np.array(self.center_data[i])[:,1]
            # 绘制当前聚类的样本散点图，s 控制大小，c 控制颜色
            plt.scatter(x,y,s=30,c=color[i])
        # 遍历每个聚类中心 c 和对应的索引 i
        for i,c in enumerate(self.center):
            # 在二维平面上绘制聚类中心，alpha 控制透明度
            plt.scatter(x=c[0],y=c[1],s=200,c=color[i],alpha=0.2)
        # 显示图片
        plt.show()
        
    def visualization_3D(self):
        ''' 三维空间可视化'''
        # 取第一个聚类的样本子集，若样本的特征维数小于 3
        if np.array(self.center_data[0]).shape[1] < 3:
            # 则返回空值，放弃三维可视化
            return None
        # 设置画布大小
        plt.figure(figsize=(10,8))
        # 设置可选颜色：红、绿、蓝
        color = ['r','g','b']
        # 创建三维空间直角坐标系
        ax = plt.axes(projection='3d')
        # 遍历每个聚类中心的索引
        for i in self.center_data:
            # 根据索引，取该聚类中心对应的样本第一列特征
            x = np.array(self.center_data[i])[:,0]
            # 根据索引，取该聚类中心对应的样本第二列特征
            y = np.array(self.center_data[i])[:,1]
            # 根据索引，取该聚类中心对应的样本第三列特征
            z = np.array(self.center_data[i])[:,2]
            # 绘制当前聚类的样本散点图，s 控制大小，c 控制颜色
            ax.scatter3D(x,y,z,s=30,c=color[i])
        # 遍历每个聚类中心 c 和对应的索引 i
        for i,c in enumerate(self.center):
            # 在三维空间中绘制当前聚类中心，alpha 控制透明度
            ax.scatter3D(xs=c[0],ys=c[1],zs=c[2],s=200,c=color[i],alpha=0.2)
        # 显示图片
        plt.show()
        
    def fit(self,data,visualization=None):
        ''' 算法迭代过程
        
        Parameters
        ----------
        data: 存放样本特征向量的 numpy array
        visualization： 可视化选项，默认为 None，不对聚类过程做可视化
        
        Returns
        -------
        self
        '''
        # 初始化聚类中心
        self.center = self.center_init(data)
        # 循环迭代
        while True:
            # 获取聚类中心对应的样本子集
            self.center_data = self.cluster(data)
            # 当前聚类中心和样本特征向量在二维空间的可视化 
            if visualization == '2d':
                self.visualization_2D()
            # 当前聚类中心和样本特征向量在三维空间的可视化 
            if visualization == '3d':
                self.visualization_3D()
            # 保存上一次迭代的聚类中心
            old_center = self.center.copy()
            # 遍历各个聚类中心的索引
            for i in self.center_data:
                # 更新每一个聚类中心：样本子集特征向量的均值
                self.center[i] = np.mean(self.center_data[i],axis=0)
            # 循环迭代的停止条件：最近两次迭代，聚类中心的位置不再变化
            if sum([self.distance(self.center[i],old_center[i]) for i in range(self.k)]) == 0:
                break
                
        return self
        
# 加载鸢尾花数据集的特征向量
data = load_iris().data
# Kmeans 聚类
model = Kmeans(3).fit(data)
# 打印聚类中心
print(model.center)

[[6.85384615 3.07692308 5.71538462 2.05384615]
 [5.88360656 2.74098361 4.38852459 1.43442623]
 [5.006      3.428      1.462      0.246     ]]


In [17]:
data

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1],
       [5.4, 3.7, 1.5, 0.2],
       [4.8, 3.4, 1.6, 0.2],
       [4.8, 3. , 1.4, 0.1],
       [4.3, 3. , 1.1, 0.1],
       [5.8, 4. , 1.2, 0.2],
       [5.7, 4.4, 1.5, 0.4],
       [5.4, 3.9, 1.3, 0.4],
       [5.1, 3.5, 1.4, 0.3],
       [5.7, 3.8, 1.7, 0.3],
       [5.1, 3.8, 1.5, 0.3],
       [5.4, 3.4, 1.7, 0.2],
       [5.1, 3.7, 1.5, 0.4],
       [4.6, 3.6, 1. , 0.2],
       [5.1, 3.3, 1.7, 0.5],
       [4.8, 3.4, 1.9, 0.2],
       [5. , 3. , 1.6, 0.2],
       [5. , 3.4, 1.6, 0.4],
       [5.2, 3.5, 1.5, 0.2],
       [5.2, 3.4, 1.4, 0.2],
       [4.7, 3.2, 1.6, 0.2],
       [4.8, 3.1, 1.6, 0.2],
       [5.4, 3.4, 1.5, 0.4],
       [5.2, 4.1, 1.5, 0.1],
       [5.5, 4.2, 1.4, 0.2],
       [4.9, 3